In [6]:
import tensorflow
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling
import matplotlib.pyplot as plt
import numpy as np
import os
DATA_DIR = "Desktop/Master/PlantVillage/" 
# --- 1. PARAMETRELERİ AYARLAMA ---
# Veri setinizin ana klasör yolunu ayarlayın.
DATA_DIR = "PlantVillage/" 
IMG_SIZE = (128, 128)  # Görüntüleri bu boyuta yeniden boyutlandıracağız
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2  # %80 Eğitim, %20 Doğrulama (Validation) için

In [2]:
if not os.path.isdir(DATA_DIR):
    print(f"HATA: '{DATA_DIR}' klasörü bulunamadı. Lütfen yolu kontrol edin.")

### Veri Seti Nesnelerini Yükleme

In [8]:
import traceback

In [3]:
import os

for root, dirs, files in os.walk(DATA_DIR):
    for d in dirs:
        if any(ch not in 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789_-' for ch in d):
            print("⚠️ Şüpheli klasör ismi:", os.path.join(root, d))

In [5]:
print(repr(DATA_DIR))
import os
print(os.path.abspath(DATA_DIR))


'PlantVillage/'
d:\Masaüstü\Master\DL Proje\PlantVillage


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path # Pathlib'i import ettiğinizden emin olun

# ... (DATA_DIR ve diğer parametrelerinizin tanımlı olduğunu varsayıyorum)

# Eğer daha önceki denemelerden DATA_PATH_OBJECT tanımlı değilse, tanımlayın:
DATA_PATH_OBJECT = Path(DATA_DIR).resolve()

# --- 2. VERİ SETİ YOLU VE ETİKETLERİNİ OLUŞTURMA ---
print("\nDosya yolları taranıyor ve DataFrame oluşturuluyor...")
filepaths = []
labels = []

# Tüm sınıf klasörlerini dolaş (iki katmanlı yapıyı kullanarak)
for class_folder in DATA_PATH_OBJECT.iterdir():
    if class_folder.is_dir():
        class_name = class_folder.name

        # Klasör içindeki resimleri topla (*.JPG ve *.jpg)
        for image_path in class_folder.glob('*.[Jj][Pp][Gg]'):
            filepaths.append(str(image_path))
            labels.append(class_name)

df = pd.DataFrame({'filepath': filepaths, 'label': labels})

class_names = df['label'].unique().tolist()
NUM_CLASSES = len(class_names)
INPUT_SHAPE = (128, 128, 3) 

print(f"Toplam dosya: {len(df)}")
print(f"Algılanan Sınıf Sayısı: {NUM_CLASSES}")

# --- 3. VERİ KÜMELERİNİ AYIRMA (Eğitim ve Doğrulama) ---

train_df, val_df = train_test_split(
    df,
    test_size=0.2, # VALIDATION_SPLIT
    shuffle=True,
    random_state=42,
    stratify=df['label']
)

# --- 4. DATA GENERATOR İLE YÜKLEME VE ÖN İŞLEME ---

# Sadece Normalizasyon (rescale=1./255) yapacak generator
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    target_size=(128, 128), # IMG_SIZE
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,          # BATCH_SIZE
    shuffle=True,
    seed=42
)

val_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='label',
    target_size=(128, 128),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

# Kalan kodun (Model Oluşturma ve Eğitme) doğru çalışması için jeneratörleri atayın
train_ds = train_generator
val_ds = val_generator


Dosya yolları taranıyor ve DataFrame oluşturuluyor...
Toplam dosya: 20636
Algılanan Sınıf Sayısı: 15
Found 16508 validated image filenames belonging to 15 classes.
Found 4128 validated image filenames belonging to 15 classes.
